In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from variables_config_1P import (
    param_info_file, 
    plots_dir, 
    redshift_values, 
    lf_data_dir,
    colour_data_dir, 
    uvlf_limits, 
    colour_limits,
    filters,
    colour_pairs
)

def read_colour(filename):
    """Read data from file."""
    try:
        data = pd.read_csv(filename, delim_whitespace=True)
        return data
    except Exception as e:
        print(f"Error reading {filename}: {e}")
        return None

def calculate_parameter_values(param_info):
    """
    Calculate all parameter values based on LogFlag.
    Returns dictionary with values for each variation (n2, n1, 0, 1, 2).
    """
    min_val = param_info['MinVal']
    max_val = param_info['MaxVal']
    fid_val = param_info['FiducialVal']
    
    if param_info['LogFlag']:
        # Logarithmic spacing
        log_min = np.log10(min_val)
        log_max = np.log10(max_val)
        log_fid = np.log10(fid_val)
        
        # Calculate n1 and 1 values in log space
        n1_val = 10**(log_min + (log_fid - log_min)/2)
        val_1 = 10**(log_fid + (log_max - log_fid)/2)
    else:
        # Linear spacing
        n1_val = min_val + (fid_val - min_val)/2
        val_1 = fid_val + (max_val - fid_val)/2
    
    return {
        'n2': min_val,    # MinVal
        'n1': n1_val,     # Calculated intermediate value
        '0': fid_val,     # FiducialVal
        '1': val_1,       # Calculated intermediate value
        '2': max_val      # MaxVal
    }

def get_safe_name(name, filter_system_only=False):
    """
    Convert string to path-safe version and/or extract filter system.
    
    Args:
        name: String to process (e.g., "GALEX FUV" or "UV1500")
        filter_system_only: If True, returns only the filter system (e.g., "GALEX" or "UV")
    
    Returns:
        Processed string (e.g., "GALEX_FUV" or "GALEX")
    """
    # Replace spaces with underscores
    safe_name = name.replace(' ', '_')
    
    # If we only want the filter system, return the first part
    if filter_system_only:
        return safe_name.split('_')[0]
    
    return safe_name

def get_colour_dir_name(band1, band2):
    """
    Create a standardized directory name for colour plots.
    Examples:
        ("GALEX FUV", "GALEX NUV") -> "GALEX_FUV-NUV"
        ("UVM2", "SUSS") -> "UVM2-SUSS"
    """
    # Extract the relevant parts of the filter names
    if ' ' in band1:
        system1, filter1 = band1.split(' ', 1)
    else:
        system1, filter1 = band1, band1

    if ' ' in band2:
        system2, filter2 = band2.split(' ', 1)
    else:
        system2, filter2 = band2, band2
        
    # If both filters are from the same system, use shortened version
    if system1 == system2:
        return f"{get_safe_name(system1)}_{filter1}-{filter2}"
    else:
        return f"{get_safe_name(band1)}-{get_safe_name(band2)}"

In [2]:
def plot_parameter_variations_uvlf(param_num, param_info, redshift, band_type, band):
    """Plot UVLF showing all parameter variations at one redshift."""
    # This is your existing plot_all_uvlf_variations function
    param_values = calculate_parameter_values(param_info)
    
    plt.figure(figsize=(10, 8))
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    
    # Get filter system from band name
    filter_system = get_safe_name(band, filter_system_only=True)
    base_output_dir = lf_data_dir[band_type][filter_system]
    
    # Plot each variation
    for var, colour in zip(variations, colours):
        filename = os.path.join(get_safe_name(base_output_dir), redshift['label'],
                              f"UVLF_1P_p{param_num}_{var}_{get_safe_name(band)}_{redshift['label']}_{band_type}.txt")
        
        if os.path.exists(filename):
            data = pd.read_csv(filename, delimiter='\t')
            label = f"{param_info['ParamName']} = {param_values[var]:.3g}"
            plt.errorbar(data['magnitude'], data['phi'], 
                        yerr=data['phi_sigma'],
                        fmt='o-', color=colour, 
                        label=label,
                        markersize=4, capsize=2)
        else:
            print(f"File not found: {filename}")

    if plt.gca().has_data():
        plt.xlabel('Magnitude (AB)', fontsize=12)
        plt.ylabel('Number Density (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        plt.xlim(*uvlf_limits)
        plt.grid(True, which='both', linestyle='--', alpha=0.6)
        plt.title(f'{param_info["Description"]} - {param_info["ParamName"]} (p{param_num})\n'
                 f'z = {redshift["redshift"]}',
                 fontsize=14)
        plt.legend(loc='upper left', title='Parameter Values')
        plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
        
        # Updated path to match new structure
        output_path = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band), 
                                 'parameter_variations',
                                 f'UVLF_p{param_num}_{param_info["ParamName"]}_z{redshift["label"]}.pdf')
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        print(f"Saving UVLF plot: {output_path}")
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
    
    plt.close()

def plot_parameter_variations_colour(param_num, param_info, redshift, band_type, colour_pairs):
    """Plot colour distribution showing all parameter variations at one redshift."""
    param_values = calculate_parameter_values(param_info)
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    
    for band1, band2 in colour_pairs:
        plt.figure(figsize=(10, 8))
        
        # Use new function for directory name
        colour_dir = get_colour_dir_name(band1, band2)
        
        for var, colour in zip(variations, colours):
            # Create consistent filename
            colour_name = f"{band1.split()[-1]}-{band2.split()[-1]}"  # Keep original naming for files
            
            filename = os.path.join(colour_data_dir[band_type][colour_dir], 
                                  redshift['label'],
                                  f"Colour_1P_p{param_num}_{var}_{colour_name}_{redshift['label']}_{band_type}.txt")
            
            if os.path.exists(filename):
                data = pd.read_csv(filename, delimiter='\t')
                plt.plot(data['colour'], data['distribution'], 
                        color=colour, linewidth=2,
                        label=f'{param_info["ParamName"]} = {param_values[var]:.3g}')
        
        plt.xlabel(f'{band1} - {band2} [mag]', fontsize=14)
        plt.ylabel('Normalized Count', fontsize=14)
        plt.xlim(*colour_limits)
        plt.grid(True, which='both', linestyle='--', alpha=0.3)
        plt.title(f'{param_info["Description"]} - {param_info["ParamName"]} (p{param_num})\n'
                 f'z = {redshift["redshift"]}', 
                 fontsize=16)
        plt.legend(title='Parameter Values', 
                  title_fontsize=12,
                  fontsize=11,
                  loc='upper right',
                  framealpha=0.95)
        
        plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
        plt.tight_layout()
        
        # Updated output path using new directory name
        output_dir = os.path.join(plots_dir, 'colours', band_type, colour_dir, 
                                'parameter_variations')
        
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, 
                                  f'Colour_p{param_num}_{param_info["ParamName"]}_z{redshift["label"]}.pdf')
        plt.savefig(output_path, bbox_inches='tight', dpi=300)
        plt.close()

In [3]:

def process_all_parameters():
    """Process all parameter variation plots."""
    colour_pairs = colour_pairs
    
    params_df = pd.read_csv(param_info_file)
    
    for param_num in range(1, 29):
        param_info = params_df.iloc[param_num-1]
        print(f"\nProcessing parameter {param_num}: {param_info['ParamName']}")
        
        for snap, redshift in redshift_values.items():
            print(f"Processing redshift {redshift['redshift']}")
            
            for band_type in ['intrinsic', 'attenuated']:
                # Process UVLFs
                for band in filters[band_type]:
                    print(f"Plotting UVLF variations for {band_type} {band}")
                    plot_parameter_variations_uvlf(param_num, param_info, redshift, band_type, band)
                
                # Process colours
                print(f"Processing colour variations for z={redshift['redshift']}")
                plot_parameter_variations_colour(param_num, param_info, redshift, band_type, colour_pairs)

In [4]:
def create_redshift_evolution_plot(param_num, param_info, band_type, band):
    """Create plot showing redshift evolution for each variation of a parameter."""
    param_values = calculate_parameter_values(param_info)
    
    variations = ['n2', 'n1', '1', '2']
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Redshift Evolution - {param_info["ParamName"]} (p{param_num})\n{param_info["Description"]}', 
                 fontsize=16, y=1.02)
    
    redshift_colours = {
        '044': 'darkred',     # z=2.00
        '052': 'orangered',   # z=1.48
        '060': 'orange',      # z=1.05
        '086': 'gold'         # z=0.10
    }
    
    filter_system = get_safe_name(band, filter_system_only=True)
    base_output_dir = lf_data_dir[band_type][filter_system]
    
    for ax, variation in zip(axes.flatten(), variations):
        for snap, redshift in redshift_values.items():
            filename = os.path.join(get_safe_name(base_output_dir), 
                                  redshift['label'],
                                  f"UVLF_1P_p{param_num}_{variation}_{get_safe_name(band)}_{redshift['label']}_{band_type}.txt")
            
            if os.path.exists(filename):
                try:
                    data = pd.read_csv(filename, delimiter='\t')
                    
                    ax.errorbar(data['magnitude'], data['phi'], 
                              yerr=data['phi_sigma'],
                              fmt='o-', color=redshift_colours[snap],
                              label=f'z = {redshift["redshift"]}',
                              markersize=4, capsize=2)
                except Exception as e:
                    print(f"Error plotting {filename}: {e}")
        
        ax.set_xlim(-25, -16)
        ax.grid(True, which='both', linestyle='--', alpha=0.3)
        ax.set_title(f'{param_info["ParamName"]} = {param_values[variation]:.3g}', 
                    fontsize=12)
        ax.set_xlabel('Magnitude (AB)', fontsize=12)
        ax.set_ylabel('Φ (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.legend(fontsize=10, title='Redshift', title_fontsize=10)
    
    plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
    plt.tight_layout()
    
    output_dir = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band), 'redshift_variations')
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 
                              f'UVLF_redshift_evolution_p{param_num}_{get_safe_name(param_info["ParamName"])}.pdf')
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()

def create_parameter_variation_plot(param_num, param_info, band_type, band):
    """Create plot showing parameter variations at each redshift."""
    param_values = calculate_parameter_values(param_info)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Parameter Variations - {param_info["ParamName"]} (p{param_num})\n{param_info["Description"]}', 
                 fontsize=16, y=1.02)
    
    variation_colours = {
        'n2': 'blue',
        'n1': 'green',
        '1': 'red',
        '2': 'purple'
    }
    
    filter_system = get_safe_name(band, filter_system_only=True)
    base_output_dir = lf_data_dir[band_type][filter_system]
    
    for ax, (snap, redshift) in zip(axes.flatten(), redshift_values.items()):
        for var in ['n2', 'n1', '1', '2']:
            filename = os.path.join(get_safe_name(base_output_dir), 
                                  redshift['label'],
                                  f"UVLF_1P_p{param_num}_{var}_{get_safe_name(band)}_{redshift['label']}_{band_type}.txt")
            
            if os.path.exists(filename):
                try:
                    data = pd.read_csv(filename, delimiter='\t')
                    label = f'{param_info["ParamName"]} = {param_values[var]:.3g}'
                    
                    ax.errorbar(data['magnitude'], data['phi'], 
                              yerr=data['phi_sigma'],
                              fmt='o-', color=variation_colours[var],
                              label=label,
                              markersize=4, capsize=2)
                except Exception as e:
                    print(f"Error plotting {filename}: {e}")
        
        ax.set_xlim(-25, -16)
        ax.grid(True, which='both', linestyle='--', alpha=0.3)
        ax.set_title(f'z = {redshift["redshift"]}', fontsize=12)
        ax.set_xlabel('Magnitude (AB)', fontsize=12)
        ax.set_ylabel('Φ (Mpc$^{-3}$ mag$^{-1}$)', fontsize=12)
        ax.tick_params(axis='both', which='major', labelsize=10)
        ax.legend(fontsize=9, title='Parameter Values', title_fontsize=10)
    
    plt.figtext(0.02, 0.02, f"LogFlag: {param_info['LogFlag']}", fontsize=8)
    plt.tight_layout()
    
    output_dir = os.path.join(plots_dir, 'LFs', band_type, get_safe_name(band), 'parameter_variations')
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, 
                              f'UVLF_parameter_variation_p{param_num}_{get_safe_name(param_info["ParamName"])}.pdf')
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()

In [5]:
# process redshift and parameter variation plots. (4 plots per fig)
def process_all_variations():
    """Process both types of variation plots for all parameters and bands."""
    params_df = pd.read_csv(param_info_file)
    
    for param_num in range(1, 29):
        param_info = params_df.iloc[param_num-1]
        print(f"\nProcessing parameter {param_num}: {param_info['ParamName']}")
        
        for band_type in ['intrinsic', 'attenuated']:
            for band in filters[band_type]:
                print(f"Creating variation plots for {band_type} {band}")
                create_redshift_evolution_plot(param_num, param_info, band_type, band)
                create_parameter_variation_plot(param_num, param_info, band_type, band)

if __name__ == "__main__":
    process_all_variations()


Processing parameter 1: Omega0
Creating variation plots for intrinsic UV1500
Creating variation plots for intrinsic GALEX FUV
Creating variation plots for intrinsic GALEX NUV
Creating variation plots for attenuated GALEX FUV
Creating variation plots for attenuated GALEX NUV

Processing parameter 2: sigma8
Creating variation plots for intrinsic UV1500
Creating variation plots for intrinsic GALEX FUV
Creating variation plots for intrinsic GALEX NUV
Creating variation plots for attenuated GALEX FUV
Creating variation plots for attenuated GALEX NUV

Processing parameter 3: WindEnergyIn1e51erg
Creating variation plots for intrinsic UV1500
Creating variation plots for intrinsic GALEX FUV
Creating variation plots for intrinsic GALEX NUV
Creating variation plots for attenuated GALEX FUV
Creating variation plots for attenuated GALEX NUV

Processing parameter 4: RadioFeedbackFactor
Creating variation plots for intrinsic UV1500
Creating variation plots for intrinsic GALEX FUV
Creating variation 

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.



Processing parameter 15: VariableWindSpecMomentum
Creating variation plots for intrinsic UV1500


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Creating variation plots for intrinsic GALEX FUV


...

Creating variation plots for intrinsic GALEX NUV


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Creating variation plots for attenuated GALEX FUV


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Creating variation plots for attenuated GALEX NUV

Processing parameter 16: WindFreeTravelDensFac
Creating variation plots for intrinsic UV1500
Creating variation plots for intrinsic GALEX FUV
Creating variation plots for intrinsic GALEX NUV
Creating variation plots for attenuated GALEX FUV
Creating variation plots for attenuated GALEX NUV

Processing parameter 17: MinWindVel
Creating variation plots for intrinsic UV1500
Creating variation plots for intrinsic GALEX FUV
Creating variation plots for intrinsic GALEX NUV
Creating variation plots for attenuated GALEX FUV
Creating variation plots for attenuated GALEX NUV

Processing parameter 18: WindEnergyReductionFactor
Creating variation plots for intrinsic UV1500
Creating variation plots for intrinsic GALEX FUV
Creating variation plots for intrinsic GALEX NUV
Creating variation plots for attenuated GALEX FUV
Creating variation plots for attenuated GALEX NUV

Processing parameter 19: WindEnergyReductionMetallicity
Creating variation plots

In [6]:
def create_lf_all(params_df, redshift, band_type, band_or_colour_pairs):
    """
    Create grid plot of all parameters for a given redshift for LFs.
    """
    nrows, ncols = 7, 4  # 28 parameters in a 7x4 grid
    fig, axes = plt.subplots(nrows, ncols, figsize=(20, 35))
    
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    
    for param_num in range(1, 29):
        row = (param_num - 1) // ncols
        col = (param_num - 1) % ncols
        ax = axes[row, col]
        
        param_info = params_df.iloc[param_num-1]
        param_values = calculate_parameter_values(param_info)
        
        if band_or_colour_pairs == "GALEX":
            filter_system = "GALEX"
            data_dir = os.path.join(lf_data_dir[band_type][filter_system], get_safe_name(redshift['label']))
        else:
            filter_system = get_safe_name(band_or_colour_pairs, filter_system_only=True)
            data_dir = os.path.join(lf_data_dir[band_type][filter_system], get_safe_name(redshift['label']))
        
        for var, colour in zip(variations, colours):
            filename = os.path.join(data_dir, f"UVLF_1P_p{param_num}_{var}_{get_safe_name(band_or_colour_pairs)}_{get_safe_name(redshift['label'])}_{band_type}.txt")
            
            print(f"Checking UVLF file: {filename}")
            
            if os.path.exists(filename):
                try:
                    data = pd.read_csv(filename, delimiter='\t')
                    print(f"UVLF data columns: {data.columns}")
                    ax.plot(data['magnitude'], data['phi'], color=colour, linewidth=1.5,
                           label=f'{param_values[var]:.3g}')
                except (FileNotFoundError, pd.errors.EmptyDataError, pd.errors.ParserError) as e:
                    print(f"Error reading UVLF file: {filename}")
                    print(e)
            else:
                print(f"UVLF file not found: {filename}")
    
    # Customize subplot
    ax.set_xlim(-25, -16)
    ax.set_ylim(1e-7, 1e-3)
    ax.grid(True, which='both', linestyle='--', alpha=0.3)
    ax.set_title(f'p{param_num}: {param_info["ParamName"]}', fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=6)
    
    # Add legend to first subplot only
    if param_num == 1:
        ax.legend(fontsize=6, title='Values', title_fontsize=6)
    
    # Add LogFlag info
    ax.text(0.02, 0.98, f'LogFlag: {param_info["LogFlag"]}',
            transform=ax.transAxes, fontsize=6, va='top')
    
    # Add overall title
    if band_or_colour_pairs == "GALEX":
        title = f'GALEX UVLF Parameter Variations (z = {redshift["redshift"]})'
    else:
        title = f'{get_safe_name(band_or_colour_pairs)} UVLF Parameter Variations (z = {redshift["redshift"]})'
    fig.suptitle(title, fontsize=16, y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    
    # Save plot
    os.makedirs(data_dir, exist_ok=True)
    plt.savefig(os.path.join(data_dir, 'All_UVLF_parameters.pdf'), bbox_inches='tight', dpi=300)
    plt.close()
    

In [14]:
def create_colour_all(params_df, redshift, band_type, band_or_colour_pairs):
    """
    Create grid plot of all parameters for a given redshift for colors.
    """
    ncols, nrows = 7, 4  # 28 parameters in a 7x4 grid
    fig, axes = plt.subplots(nrows, ncols, figsize=(35, 20))
    
    variations = ['n2', 'n1', '1', '2']
    colours = ['blue', 'green', 'red', 'purple']
    
    for param_num in range(1, 29):
        col = (param_num - 1) // nrows
        row = (param_num - 1) % nrows
        ax = axes[row, col]
        
        param_info = params_df.iloc[param_num-1]
        param_values = calculate_parameter_values(param_info)
        
        band1, band2 = band_or_colour_pairs[0]  # Assuming single colour pair
        filter_system = get_colour_dir_name(band1, band2)
        
        data_dir = os.path.join(colour_data_dir[band_type][filter_system], get_safe_name(redshift['label']))
        
        for var, colour in zip(variations, colours):
            filename = os.path.join(data_dir, f"Colour_1P_p{param_num}_{var}_{filter_system}_{get_safe_name(redshift['label'])}_{band_type}.txt")
            print(f"Checking color file: {filename}")
            
            if os.path.exists(filename):
                try:
                    data = pd.read_csv(filename, delimiter='\t')
                    print(f"Color data columns: {data.columns}")
                    ax.plot(data['colour'], data['distribution'], color=colour, linewidth=1.5,
                           label=f'{param_values[var]:.3g}')
                except (FileNotFoundError, pd.errors.EmptyDataError, pd.errors.ParserError) as e:
                    print(f"Error reading color file: {filename}")
                    print(e)
            else:
                print(f"Color file not found: {filename}")
                
    # Customize subplots
    for ax in axes.flatten():
        ax.set_xlim(*colour_limits)
        ax.grid(True, which='both', linestyle='--', alpha=0.3)
        ax.tick_params(axis='both', which='major', labelsize=6)
    
    # Add parameter labels and LogFlag info
    for param_num, ax in enumerate(axes.flatten(), start=1):
        param_info = params_df.iloc[param_num-1]
        ax.set_title(f'p{param_num}: {param_info["ParamName"]}', fontsize=8)
        ax.text(0.02, 0.98, f'LogFlag: {param_info["LogFlag"]}',
                transform=ax.transAxes, fontsize=6, va='top')
    
    # Add color bar
    cmap = mpl.colors.ListedColormap(colours)
    norm = mpl.colors.BoundaryNorm([-2.5, -1.5, -0.5, 0.5, 1.5], cmap.N)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar_ax = fig.add_axes([0.375, 0.95, 0.25, 0.01])  # change position and size here
    cbar = fig.colorbar(sm, cax=cbar_ax, orientation='horizontal', ticks=[-2, -1, 0, 1])
    cbar.ax.set_xticklabels(['Min', '', '0', 'Max'])
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label('Param values', fontsize=14)
    
    # Add overall title
    title = f'{filter_system} Colour Distribution {param_info["ParamName"]} Variations (z = {redshift["redshift"]})'
    fig.suptitle(title, fontsize=16, y=0.99)
    
    # Adjust subplot spacing to make room for color bar
    plt.subplots_adjust(top=0.93, bottom=0.05, hspace=0.3, wspace=0.2)
    
    # Save plot
    plot_dir = os.path.join(plots_dir, 'colours', band_type, filter_system, get_safe_name(redshift['label']))
    os.makedirs(plot_dir, exist_ok=True)
    plt.savefig(os.path.join(plot_dir, f"All_colours_{filter_system}_{get_safe_name(redshift['label'])}_{band_type}.pdf"), bbox_inches='tight', dpi=300)
    plt.close()

In [15]:
def process_lfs_all(redshift_values, filters):
    """Process UVLF parameter grids for all redshifts."""
    params_df = pd.read_csv(param_info_file)
    
    for snap, redshift_info in redshift_values.items():
        print(f"\nProcessing redshift {redshift_info['redshift']}")
        
        for band_type, band_list in filters.items():
            for band in band_list:
                print(f"Creating UVLF parameter grid for {band_type} {band}")
                create_lf_all(
                    params_df=params_df,
                    redshift=redshift_info,
                    band_type=band_type,
                    band_or_colour_pairs=band
                )


def process_colours_all(redshift_values, colour_pairs):
    """Process color parameter grids for all redshifts."""
    params_df = pd.read_csv(param_info_file)
    
    for snap, redshift_info in redshift_values.items():
        print(f"\nProcessing redshift {redshift_info['redshift']}")
        
        for band_type in ['intrinsic', 'attenuated']:
            print(f"Creating colour parameter grid for {band_type}")
            create_colour_all(
                params_df=params_df,
                redshift=redshift_info,
                band_type=band_type,
                band_or_colour_pairs=colour_pairs
            )

if __name__ == "__main__":
    
    #process_lfs_all(redshift_values, filters)
    process_colours_all(redshift_values, colour_pairs)


Processing redshift 2.0
Creating colour parameter grid for intrinsic
Checking color file: /disk/xray15/aem2/data/28pams/IllustrisTNG/1P/colours/intrinsic/GALEX_FUV-NUV/z2.0/Colour_1P_p1_n2_GALEX_FUV-NUV_z2.0_intrinsic.txt
Color data columns: Index(['colour', 'distribution'], dtype='object')
Checking color file: /disk/xray15/aem2/data/28pams/IllustrisTNG/1P/colours/intrinsic/GALEX_FUV-NUV/z2.0/Colour_1P_p1_n1_GALEX_FUV-NUV_z2.0_intrinsic.txt
Color data columns: Index(['colour', 'distribution'], dtype='object')
Checking color file: /disk/xray15/aem2/data/28pams/IllustrisTNG/1P/colours/intrinsic/GALEX_FUV-NUV/z2.0/Colour_1P_p1_1_GALEX_FUV-NUV_z2.0_intrinsic.txt
Color data columns: Index(['colour', 'distribution'], dtype='object')
Checking color file: /disk/xray15/aem2/data/28pams/IllustrisTNG/1P/colours/intrinsic/GALEX_FUV-NUV/z2.0/Colour_1P_p1_2_GALEX_FUV-NUV_z2.0_intrinsic.txt
Color data columns: Index(['colour', 'distribution'], dtype='object')
Checking color file: /disk/xray15/aem2/d